# Correlation between Detected Breeding Sites and Larval Survey

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import plotly.graph_objs as go

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, graphviz
from sklearn import *

from copy import deepcopy
from scipy.stats.stats import pearsonr, spearmanr
from shapely.geometry import Polygon
from collections import Counter

loo = model_selection.LeaveOneOut()

month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
categories = np.array(['bin','bowl','bucket','cup','jar','pottedplant','tire','vase']).reshape(-1,1)

In [4]:
import visualizer
import data_loader

df_loader = data_loader.df_loader()

df_survey = df_loader.load_survey()
df_filtered = df_loader.load_filterd('bi')
df_area = df_loader.load_area()
df_detect = df_loader.load_detect()

df_population = df_loader.load_population()
df_dengue_cases = df_loader.load_cases()

In [6]:
df_survey.head()
print('Total:', len(df_survey))

,district,subdist,village,hi,ci,bi
date,,,,,,
2015-02-01,ขนอม,ขนอม,บางโหนด,16.0,2.44,15.0
2015-02-01,ฉวาง,นาแว,ห้วยทรายขาว,15.0,1.87,18.0
2015-02-01,เฉลิมพระเกียรติ,ทางพูน,สะเพรง,10.0,2.35,10.0
2015-02-01,ชะอวด,บ้านตูล,กุมแป,19.0,3.42,31.0
2015-02-01,ช้างกลาง,ช้างกลาง,หน้าเหมน,16.0,2.06,20.0


Total: 484


In [7]:
df_filtered.head()
len(df_filtered)

,district,subdist,village,hi,ci,bi
date,,,,,,
2015-02-01,ขนอม,ขนอม,บางโหนด,16.0,2.44,15.0
2015-05-01,ขนอม,ขนอม,บางโหนด,5.0,0.93,5.0
2015-02-01,ฉวาง,นาแว,ห้วยทรายขาว,15.0,1.87,18.0
2015-03-01,ฉวาง,นาแว,ไสโคกเกาะ,20.0,3.27,20.0
2015-03-01,ฉวาง,นาแว,วังโก,15.0,2.91,15.0


376

In [8]:
df_detect.head()
print('Total:', len(df_detect))

,district,subdist,bin,bowl,bucket,cup,jar,pottedplant,tire,vase,total
date,,,,,,,,,,,
2016-03-01,ร่อนพิบูลย์,ร่อนพิบูลย์,2,0,46,47,0,19,59,0,173
2016-03-01,ทุ่งสง,กะปาง,6,8,71,160,1,18,266,1,531
2016-03-01,ชะอวด,ชะอวด,5,14,46,200,7,37,329,1,639
2016-03-01,ทุ่งใหญ่,ท่ายาง,5,13,46,228,13,83,912,0,1300
2016-03-01,จุฬาภรณ์,บ้านควนมุด,2,1,101,100,3,42,142,0,391


Total: 167


In [9]:
df_area.head()
len(df_area)

,district,subdist,area
0,พรหมคีรี,ทอนหงส์,1070.58
1,ท่าศาลา,โมคลาน,341.44
2,เมืองนครศรีธรรมราช,ปากพูน,856.09
3,พิปูน,พิปูน,451.83
4,ท่าศาลา,ดอนตะโก,275.56


170

In [9]:
df_population.head()
len(df_population)

,district,subdist,n_villages,population,area
0,เมืองนครศรีธรรมราช,ในเมือง,0,43327,NaN
1,เมืองนครศรีธรรมราช,ท่าวัง,0,17346,NaN
2,เมืองนครศรีธรรมราช,คลัง,0,17568,NaN
3,เมืองนครศรีธรรมราช,ท่าไร่,6,8224,48.83
4,เมืองนครศรีธรรมราช,ปากนคร,6,15137,55.83


169

## Visualize average breteau index value for each month

In [10]:
key, val = [], []
for m in range(12):
    key.append(month[m])
    val.append(round(df_filtered[df_filtered.index.month == m+1]['bi'].mean(),0))

trace_bar = go.Bar( 
    x = key,
    y = val,
    text = val,
    textposition = 'auto',
    marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ),
    opacity=0.8
)
layout = go.Layout(
    title='Average BI for each Month',
    height=550,
    width=750,
    yaxis= dict(title='avg bi'),
    xaxis= dict(title='month')
)
fig = go.Figure(data=[trace_bar], layout=layout)
iplot(fig)

## Visualize number of detected breeding sites (containers)  for each month

In [11]:
key, val = [], []
for m in range(12):
    key.append(month[m])
    val.append(round(df_detect[df_detect.index.month == m+1]['total'].mean(),0))

trace_bar = go.Bar( 
    x = key,
    y = val,
    text = val,
    textposition = 'auto',
    marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ),
    opacity=0.8
)
layout = go.Layout(
    title='Number of detected breeding sites for each Month',
    height=550,
    width=750,
    yaxis= dict(title='Number of containers'),
    xaxis= dict(title='month')
)
fig = go.Figure(data=[trace_bar], layout=layout)
iplot(fig)

In [11]:
# trace = go.Scatter(
#     x = df_area['area'], y = df_population['area'], mode = 'markers',
#     marker = dict(size = 12, opacity = 0.3)
# )
# iplot(go.Figure(data=[trace]))

# Correlation
Perform correlation between number of detected containers and breteau index

## Filter outliers in each class

In [12]:
# df_category_stat = []
# for column in np.squeeze(categories):
#     mean_det, std_det = df_detect[column].mean(), df_detect[column].std()
#     df_category_stat.append([column, mean_det, std_det])
    
# df_category_stat = pd.DataFrame.from_records(df_category_stat)
# df_category_stat.columns = ['class','mean','std']
# df_category_stat = df_category_stat.set_index('class')
# df_category_stat

## Regression Models

In [13]:
def plot_regression(regr, name):
    
    regr.fit(X, y)
    
    try: coef = regr.coef_.reshape(-1,1)
    except: pass

    try: coef = regr.feature_importances_.reshape(-1,1)
    except: pass
    
#     df_coef = pd.DataFrame.from_records(np.concatenate((features_name, coef), axis=1))
#     df_coef.columns = ['class', 'coef']
#     df_coef['coef'] = df_coef['coef'].astype(float).round(4)
    
    y_pred, y_true = [], []
    for train_index, test_index in loo.split(X):

        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]

        _=regr.fit(X_train, Y_train)

        pred = regr.predict(X_test)
        y_pred.append(np.squeeze(pred))
        y_true.append(np.squeeze(Y_test))

    y_pred = np.array(y_pred)
    y_true = np.array(y_true)

    print('R-squared:', metrics.r2_score(y_true, y_pred))
    print('Person:', pearsonr(y_true, y_pred))
    print(spearmanr(y_true, y_pred),'\n')

    trace_1 = go.Scatter(
        x = y_true, y = y_pred, mode = 'markers', name='Scatter',
        marker = dict(size = 12, opacity = 0.5)
    )
    
    xs = np.array(y_true)
    ys = np.array(y_pred)

    regr = linear_model.LinearRegression()
    regr.fit(xs.reshape(-1, 1), ys.reshape(-1, 1))

    ys_pred = regr.predict(xs.reshape(-1, 1))
    trace_2 = go.Scatter(
        x = xs, y = np.squeeze(ys_pred), name='Regression',
        mode = 'lines', line = dict(width = 4)
    )

#     coef = np.squeeze(coef)
#     coef = 100.0 * (coef / coef.max())
#     sorted_idx = np.argsort(coef)[::-1]
#     pos = np.arange(sorted_idx.shape[0]) + .5
    
#     trace_3 = go.Bar(
#         x = np.squeeze(features_name[sorted_idx]),
#         y = coef[sorted_idx],
#         name='Variable Importance',
#         marker=dict(
#                     color='rgb(158,202,225)',
#                     line=dict(
#                         color='rgb(8,48,107)',
#                         width=1.5),
#                 ),
#         opacity=0.8
#     )

    
    fig = tools.make_subplots(
        rows=1, cols=1, 
#         subplot_titles=(
#             'Correlation Subplot',
#             'Variable Importance Subplot',
#         ),
        vertical_spacing = 0.15
    )

    fig.append_trace(trace_1, 1, 1)
    fig.append_trace(trace_2, 1, 1)
#     fig.append_trace(trace_3, 1, 1)
    
    fig['layout'].update(width=650,title=name, font=dict(size=16))
    fig['layout']['xaxis1'].update(title='Breteau index')
#     fig['layout']['xaxis1'].update(title='Breeding site')
    
    fig['layout']['yaxis1'].update(title='Predicted')
#     fig['layout']['yaxis1'].update(title='Relative Importance')
    
    iplot(fig)
    
#     return df_coef

In [14]:
def plot_one_regression(regr, name):
    
    regr.fit(X, y)   
    
    y_pred, y_true = [], []
    for train_index, test_index in loo.split(X):

        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]

        _=regr.fit(X_train, Y_train)

        pred = regr.predict(X_test)
        y_pred.append(np.squeeze(pred))
        y_true.append(np.squeeze(Y_test))

    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    
    r2 = round(metrics.r2_score(y_true, y_pred),4)
    pearson = round(pearsonr(y_true, y_pred)[0],4)
    spearman = round(spearmanr(y_true, y_pred)[0],4)
    
    print('R-squared:', metrics.r2_score(y_true, y_pred))
    print('Person:', pearsonr(y_true, y_pred))
    print(spearmanr(y_true, y_pred),'\n')

    trace_1 = go.Scatter(
        x = y_true, y = y_pred, mode = 'markers', name='Scatter',
        marker = dict(size = 12, opacity = 0.5)
    )
    
    xs = np.array(y_true)
    ys = np.array(y_pred)

    regr = linear_model.LinearRegression()
    regr.fit(xs.reshape(-1, 1), ys.reshape(-1, 1))

    ys_pred = regr.predict(xs.reshape(-1, 1))
    trace_2 = go.Scatter(
        x = xs, y = np.squeeze(ys_pred), name='Regression',
        mode = 'lines', line = dict(width = 4)
    )

    
    fig = tools.make_subplots(
        rows=1, cols=1
    )

    fig.append_trace(trace_1, 1, 1)
    fig.append_trace(trace_2, 1, 1)
    
    name += 'R-squared: ' + str(r2) + \
    ', Pearson: ' + str(pearson) + \
    ', Spearman: ' + str(spearman)
    
    fig['layout'].update(width=650,title=name, font=dict(size=16))
    fig['layout']['xaxis1'].update(title='Breteau index')
    
    fig['layout']['yaxis1'].update(title='Predicted')
    
    iplot(fig)

## Selecte Features Used

In [13]:
# features_name = np.concatenate((categories,[['month']]), axis=0)    
# features_name = np.concatenate((categories,[['month'], ['area'], ['popluation']]), axis=0)    
# features_name = np.array([['bucket'], ['jar'], ['pottedplant'], ['month'], ['area'], ['popluation']])
# features_name = np.array([['bucket'], ['jar'], ['pottedplant']])
# features_name = np.array([['bucket'], ['jar'], ['pottedplant'], ['month']])

features_name = np.array([['bin'], ['bowl'], ['bucket'], ['jar'], ['pottedplant'], ['tire']])

# features_name = deepcopy(categories)

features_name
features_name.shape

array([['bin'],
       ['bowl'],
       ['bucket'],
       ['jar'],
       ['pottedplant'],
       ['tire']], dtype='<U11')

(6, 1)

In [16]:
df_detect['brd_total'] = df_detect['total']-df_detect['cup']-df_detect['bowl']
df_detect.head()

,district,subdist,bin,bowl,bucket,cup,jar,pottedplant,tire,vase,total,brd_total
date,,,,,,,,,,,,
2016-03-01,ร่อนพิบูลย์,ร่อนพิบูลย์,2,0,59,46,47,19,0,0,173,127
2016-03-01,ทุ่งสง,กะปาง,6,1,266,71,160,18,1,8,531,459
2016-03-01,ชะอวด,ชะอวด,5,1,329,46,200,37,7,14,639,592
2016-03-01,ทุ่งใหญ่,ท่ายาง,5,0,912,46,228,83,13,13,1300,1254
2016-03-01,จุฬาภรณ์,บ้านควนมุด,2,0,142,101,100,42,3,1,391,290


In [24]:
x_train, y_train = [], [] 
xs, ys = [], []

column = 'total'

mean_det, std_det = df_detect[column].mean(), df_detect[column].std()

subdist_list = df_survey['subdist'].unique()
for subdist in subdist_list:
    detect = round(df_detect.loc[df_detect['subdist'] == subdist][column].mean(),2)
    area = round(df_area.loc[df_area['subdist'] == subdist]['area'].mean(),2)
    
    population = round(df_population.loc[df_population['subdist'] == subdist]['population'].mean(),2)
    n_villages = round(df_population.loc[df_population['subdist'] == subdist]['n_villages'].mean(),2)
    
    survey = round(df_filtered.loc[(df_filtered['subdist'] == subdist) 
#                                    & (df_filtered.index.month.isin([6,7,8,9,10,11]))
                                  ]['bi'].mean(), 2)

    if np.isnan(detect) or np.isnan(survey): continue
    if detect > mean_det+1*std_det or detect < mean_det-1*std_det: continue
    
    xs.append(survey)
    ys.append((detect)/(area))
    
    x = df_detect.loc[df_detect['subdist'] == subdist].copy()
#     x = x[['bin','bowl','bucket','cup','jar','pottedplant','tire','vase']].copy()
#     x = x[['bin','bowl','bucket', 'jar','pottedplant','tire']].copy()

    x = x[['bucket','jar','pottedplant']].copy()

   
    month = df_detect.loc[df_detect['subdist'] == subdist].index.month[0]
    
    y_dengue_season = 1 if month in [6,7,8,9,10] else 0
    n_dengue_season = 0 if month in [6,7,8,9,10] else 1

    features = list(np.squeeze(x.values)/area) + [month]
#     features = list(np.squeeze(x.values))
#     features = np.array(detect/area)
    
    x_train.append(np.array(features))
    y_train.append(survey)
    

X = np.array(x_train)
y = np.array(y_train)
print('X_train.shape:', X.shape)

print('\nR-squared:', metrics.r2_score(xs, ys))
print('Person:', pearsonr(xs, ys))
print(spearmanr(xs, ys),'\n')

trace = go.Scatter(
    x = xs, y = ys, mode = 'markers', name='Subdistrict',
    marker = dict(size = 15, opacity = 0.4)
)

xs = np.array(xs)
ys = np.array(ys)

regr = linear_model.LinearRegression()
regr.fit(xs.reshape(-1, 1), ys.reshape(-1, 1))

ys_pred = regr.predict(xs.reshape(-1, 1))
trace_2 = go.Scatter(
    x = xs, y = np.squeeze(ys_pred), mode = 'lines', line = dict(width = 4), name='Regression'
)

layout = dict(
    title = 'Entire year (53 data points): Linear Regression<br>' \
            'Pearson: 0.350, Spearman: 0.358',
    width=650, 
    xaxis = dict(title = 'Breteau index'),
    yaxis = dict(title = 'Total # of detected containers<br>divided by area'),
    font=dict(size=16)
)
iplot(go.Figure(data=[trace,trace_2], layout=layout))

X_train.shape: (53, 4)

R-squared: -2.944187145101225
Person: (0.3415996038632038, 0.01230101858605397)
SpearmanrResult(correlation=0.35785145010693775, pvalue=0.008517807331374678) 



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
regr.fit(ys.reshape(-1, 1), xs.reshape(-1, 1))
pred = np.squeeze(regr.predict(ys.reshape(-1, 1)))

print('\nR-squared:', metrics.r2_score(xs, pred))
print('Person:', pearsonr(xs, pred))
print(spearmanr(xs, pred),'\n')

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)


R-squared: 0.11669028935949799
Person: (0.34159960386320365, 0.012301018586054015)
SpearmanrResult(correlation=0.35785145010693775, pvalue=0.008517807331374678) 



In [21]:
# parameter_grid_gb = {
#     'max_depth': [3, 4, 5, 6, 7, 8],
#     'max_features': [2, 3, 4],
#     'subsample': [0.6, 0.8, 1],
#     'learning_rate':[0.03, 0.05, 0.1]
# }

# parameter_grid_tree = {
#     'max_depth': [3, 4, 5, 6, 7, 8],
#     'max_features': [2, 3, 4],
# }

# parameter_grid_svr = {
#     'kernel': ['linear','poly','rbf'],
#     'degree': [1,2,3,4,5,6]
# }


# parameter_grid_ada = {
#     'base_estimator': [svr, dt],
#     'n_estimators': [5, 10, 15, 20, 25],
#     'loss': ['linear', 'square', 'exponential'],
#     'learning_rate':[0.1]
# }

In [26]:
# grid_search = model_selection.GridSearchCV(
#     estimator=ensemble.GradientBoostingRegressor(),
#     param_grid=parameter_grid_gb, 
#     cv=loo,
#     n_jobs=8
# )

# _=grid_search.fit(X, y)
# grid_search.best_score_
# grid_search.best_params_


# grid_search = model_selection.GridSearchCV(
#     estimator=ensemble.RandomForestRegressor(), 
#     param_grid=parameter_grid_tree, 
#     cv=loo,
#     n_jobs=1
# )

# _=grid_search.fit(X, y)
# grid_search.best_score_
# grid_search.best_params_

# grid_search = model_selection.GridSearchCV(estimator=svm.SVR(), 
#                                            param_grid=parameter_grid_svr, 
#                                            cv=10,
#                                            n_jobs=8)

# _=grid_search.fit(X, y)
# grid_search.best_score_
# grid_search.best_params_

In [ ]:
X = X.reshape(-1,1)

In [27]:
X[0]
X.shape

array([0.73759296, 0.44316285, 0.0880255 , 4.        ])

(53, 4)

In [19]:
svr = svm.SVR(kernel='poly',  degree=3)
rf = ensemble.RandomForestRegressor(max_depth=3, max_features=3)
dt = tree.DecisionTreeRegressor(max_depth=3, max_features=3)
gb = ensemble.GradientBoostingRegressor(learning_rate=0.00/9, max_depth=3, max_features=3, subsample=1)

linear = linear_model.LinearRegression()
bayes = linear_model.BayesianRidge()
knn = neighbors.KNeighborsRegressor()

bag_rf = ensemble.BaggingRegressor(rf)
bag_svr = ensemble.BaggingRegressor(svr)

ada = ensemble.AdaBoostRegressor()
ada_svr = ensemble.AdaBoostRegressor(svm.NuSVR(kernel='poly', degree=3, tol=12.3, gamma=0.28), learning_rate=0.001, loss='linear')
ada_dt = ensemble.AdaBoostRegressor(dt, learning_rate=0.03, loss='linear')
ada_rf = ensemble.AdaBoostRegressor(rf, learning_rate=0.03, loss='linear')

regrs = [
#     [linear, 'Linear Regression'],
#     [svm.NuSVR(kernel='poly', degree=3, tol=12.3, gamma=0.3), 'NuSVR'],
#     [svm.SVR(kernel='poly', degree=3, tol=1), 'SVR'],
#     [bayes, 'Bayesian Ridge'],
#     [rf, 'Random Forest'],
#     [dt, 'Decision Tree'],
#     [gb, 'Gradient Boosting'],
#     [ada_svr, 'Ada SVR'],
#     [ada_dt, 'Ada DT'],
#     [ada_rf, 'Ada RF'],
#     [knn, 'KNeighbors'],
#     [bag_rf, 'Bagging RF'],
#     [bag_svr, 'Bagging SVR']
]

In [28]:
regrs = [
#     [linear, 'Linear Regression'],
    [svm.NuSVR(kernel='poly', degree=3, tol=12.3, gamma=0.28), 'NuSVR'],
#     [svm.SVR(kernel='poly', degree=3, tol=1), 'SVR'],
#     [bayes, 'Bayesian Ridge'],
#     [rf, 'Random Forest'],
#     [dt, 'Decision Tree'],
#     [gb, 'Gradient Boosting'],
#     [ada_svr, 'Ada SVR'],
]

df_selection = []
for k in range(1):
    df_compare = []
    for regr, name in regrs:
        y_pred, y_true = [], []
        for train_index, test_index in loo.split(X):
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = y[train_index], y[test_index]
            _=regr.fit(X_train, Y_train)
            pred = regr.predict(X_test)
            y_true.append(np.squeeze(Y_test))
            y_pred.append(np.squeeze(pred))

        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

        df_compare.append([
            name+'-'+str(k+1),
            metrics.r2_score(y_true, y_pred),
            pearsonr(y_true, y_pred)[0],
            spearmanr(y_true, y_pred)[0]
        ])

    df_compare = pd.DataFrame.from_records(df_compare)
    df_compare.columns = ['Model','R-squared','Pearson','Spearman']
    df_compare = df_compare.set_index('Model')
    df_compare = df_compare.round(4)
    df_selection.append(df_compare)
df_selection = pd.concat(df_selection, axis=0)

tmp = pd.DataFrame([[df_selection['R-squared'].mean(), 
                     df_selection['Pearson'].mean(), 
                     df_selection['Spearman'].mean()]])
tmp.columns = ['R-squared','Pearson','Spearman']
tmp.index = ['Average']

df_selection = df_selection.append(tmp)
df_selection

,R-squared,Pearson,Spearman
NuSVR-1,0.181,0.4261,0.446
Average,0.181,0.4261,0.446


In [30]:
k = 0
plot_one_regression(
    regrs[k][0], 
    'Entire year ('+str(X.shape[0])+' data points): '+regrs[k][1]+'<br>'
)

R-squared: 0.18097336328228186
Person: (0.42613722141249766, 0.0014649903995768562)
SpearmanrResult(correlation=0.4459633971977265, pvalue=0.0008175740595549722) 

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [20]:
visualizer.plot_importance(regrs[2][0], regrs[2][1], X, y, loo, features_name)